### Imports

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window


### Data cleaning
A function was created to read data from S3 bucket into a dataframe.

In [0]:
def read_s3_data(mount_name):
    """
    Reads JSON data from an S3 mount point into a Spark DataFrame.

    Args:
        mount_name (str): The name of the S3 mount point.

    Returns:
        DataFrame: A Spark DataFrame containing the read JSON data.
    """

    file_location = f"/mnt/{mount_name}/*.json"
    file_type = "json"
    infer_schema = "true"
    df = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .load(file_location)
    return df


Reading the pin data and cleaning it

In [0]:
# reading the data
df_pin = read_s3_data()

# replacing invalid entries with `None`
columns_to_clean = ['category', 'description', 'follower_count', 'poster_name', 'tag_list', 'is_image_or_video', 'image_src', 'save_location', 'unique_id', 'title']

for column in columns_to_clean:
    df_pin = df_pin.replace({"": None}, subset=[column])
# replacing k and M with 000 and 000000 respectivly.
df_pin = (df_pin
    .withColumn('follower_count', 
        when(df_pin.follower_count.endswith('k'), regexp_replace(df_pin.follower_count, 'k', '000'))
        .when(df_pin.follower_count.endswith('M'), regexp_replace(df_pin.follower_count, 'M', '000000'))
        .otherwise(df_pin.follower_count))
)
# casting follower count to integers
df_pin = df_pin.withColumn('follower_count', df_pin.follower_count.cast('int'))

# renaming index to ind.
df_pin = df_pin.withColumnRenamed('index', 'ind')

# adjusting the 'save_location' column to display the path
df_pin = df_pin.withColumn('save_location', regexp_replace( 'save_location', 'Local save in ', ''))

# Reorder the DataFrame columns
column_order = ['ind', 'unique_id', 'title', 'description', 'follower_count', 
                'poster_name', 'tag_list', 'is_image_or_video', 'image_src', 
                'save_location', 'category']
df_pin = df_pin.select(column_order)






In [0]:
# Reorder the DataFrame columns
column_order = ['ind', 'unique_id', 'title', 'description', 'follower_count', 
                'poster_name', 'tag_list', 'is_image_or_video', 'image_src', 
                'save_location', 'category']
df_pin = df_pin.select(column_order)



Reading the geo data and cleaning it

In [0]:
# reading the data
df_geo = read_s3_data()

# Create a new column 'coordinates' containing an array based on 'latitude' and 'longitude' columns
df_geo = df_geo.withColumn('coordinates', array(df_geo.latitude, df_geo.longitude))

# Drop the 'latitude' and 'longitude' columns from the DataFrame
df_geo = df_geo.drop('latitude', 'longitude')

# converting 'timestamp' to a timestamp data type, and formatting it.
df_geo = df_geo.withColumn('timestamp', to_timestamp(df_geo.timestamp))
df_geo = df_geo.withColumn('timestamp', date_format('timestamp', 'yyyy-MM-dd HH:mm:ss'))

# reordering the dataframe columns
df_geo = df_geo.select('ind','country','coordinates','timestamp')


Reading the user data and cleaning it

In [0]:
# reading the data
df_user = read_s3_data()

# Create a new column 'user_name' by concatenating 'first_name' and 'last_name' columns
df_user = df_user.withColumn('user_name', concat_ws('', col('first_name'), col('last_name')))

# Drop the 'first_name' and 'last_name' columns from the DataFrame
df_user = df_user.drop("first_name", "last_name")

# converting 'date_joined' to a timestamp data type, and formatting it.
df_user = df_user.withColumn('date_joined', to_timestamp(df_user.date_joined))
df_user = df_user.withColumn('date_joined', date_format('date_joined', 'yyyy-MM-dd HH:mm:ss'))

# reordering columns
df_user = df_user.select('ind', 'user_name', 'age', 'date_joined')

# Ensure 'ind' and 'age' columns are casted to the appropriate data types
df_user = df_user.withColumn("ind", df_user["ind"].cast("int"))
df_user = df_user.withColumn("age", df_user["age"].cast("int"))

  

### Querying the data
Find the most popular Pinterest category people post to based on their country.



In [0]:
# Join df_pin with df_geo based on unique_id to get the country information
joined_df = df_pin.join(df_geo.select("ind", "country"), on="ind", how="inner")

# Group the data by country and category, and count the occurrences of each category within each country
category_count_df = joined_df.groupby("country", "category").agg(count("*").alias("category_count"))

# Rank the categories within each country based on the category count
windowSpec = Window.partitionBy("country").orderBy(col("category_count").desc())
ranked_category_count_df = category_count_df.withColumn("rank", row_number().over(windowSpec))

# Filter to keep only the top-ranked category for each country
most_popular_category_df = ranked_category_count_df.filter(col("rank") == 1).drop("rank")


Find how many posts each category had between 2018 and 2022.



In [0]:
# creating temporary dataframes
df_pin.createOrReplaceTempView('df_temp_pin')
df_geo.createOrReplaceTempView('df_temp_geo')

# the SQL query

#creating a CTE
cte_query = """
       WITH ranking_table AS (
              SELECT year(df_temp_geo.timestamp) AS post_year,
                     df_temp_pin.category AS category,
                     COUNT(df_temp_pin.category) AS category_count,
                     RANK() OVER (PARTITION BY year(df_temp_geo.timestamp) ORDER BY COUNT(df_temp_pin.category) DESC) AS rank
              FROM df_temp_geo
              JOIN df_temp_pin ON df_temp_geo.ind = df_temp_pin.ind  
              WHERE 2018 <= year(df_temp_geo.timestamp) AND year(df_temp_geo.timestamp) <= 2022
              GROUP BY year(df_temp_geo.timestamp), df_temp_pin.category     
)
"""

result = spark.sql(cte_query + """
                     SELECT post_year,
                           category,
                           category_count
                     FROM ranking_table
                     WHERE rank == 1
                      """)


Find the user with most followers with their corresponding country



In [0]:

# creating temporary dataframes
df_pin.createOrReplaceTempView('df_temp_pin')
df_geo.createOrReplaceTempView('df_temp_geo')

# SQL query 
cte_query = """
       WITH ranking_table AS (
              SELECT df_temp_geo.country AS country,
                     df_temp_pin.poster_name AS poster_name,
                     df_temp_pin.follower_count AS follower_count,
                     RANK() OVER(PARTITION BY df_temp_geo.country ORDER BY df_temp_pin.follower_count DESC) AS rank
              FROM df_temp_pin
              JOIN df_temp_geo ON df_temp_geo.ind = df_temp_pin.ind
       )
"""
result_2 = spark.sql(cte_query + """
                    SELECT country,
                           poster_name,
                           follower_count
                    FROM ranking_table
                    WHERE rank == 1
                    ORDER BY follower_count DESC
                      """)
                  
result_2 = spark.createDataFrame([result_2.head(1)[0]])
result_2 = result_2.drop('poster_name')   



What is the most popular category people post to based on age groups



In [0]:
  # Join df_pin with df_user based on the common column 'ind' to get the age information
  joined_df = df_pin.join(df_user.select("ind", "age"), on="ind", how="inner")

  # Categorize the age groups based on the original age column
  df_pin_age_group = joined_df.withColumn("age_group",
      when((col("age") >= 18) & (col("age") <= 24), "18-24")
      .when((col("age") >= 25) & (col("age") <= 35), "25-35")
      .when((col("age") >= 36) & (col("age") <= 50), "36-50")
      .otherwise("+50")
  )

  # Group the data by age group and category, and count the occurrences of each category within each age group
  category_count_df = df_pin_age_group.groupBy("age_group", "category").agg(count("*").alias("category_count"))


 What is the median follower count for users in different age groups



In [0]:


# Join df_user_cleaned with df_pin based on the common column 'poster_name' to get the age and follower count information
joined_df = df_user.join(df_pin.select("ind", "follower_count"), on="ind", how="inner")

# Categorize the age groups based on the original age column
df_age_group = joined_df.withColumn("age_group",
    when((col("age") >= 18) & (col("age") <= 24), "18-24")
    .when((col("age") >= 25) & (col("age") <= 35), "25-35")
    .when((col("age") >= 36) & (col("age") <= 50), "36-50")
    .otherwise("+50")
)

# Calculate the median follower count for each age group
median_follower_count_df = df_age_group.groupBy("age_group").agg(
    percentile_approx("follower_count", 0.5).alias("median_follower_count")
)



Find how many users have joined between 2015 and 2020.



In [0]:

# Join df_user with df_geo based on a common key to get the date_joined information
joined_df = df_user.join(df_geo.select("ind", "timestamp"), on="ind", how="inner")

# Extract the year from the timestamp column
joined_df = joined_df.withColumn("join_year", year("timestamp"))

# Filter the DataFrame to select users who joined between 2015 and 2020
filtered_df = joined_df.filter((col("join_year") >= 2015) & (col("join_year") <= 2020))

# Group the data by join year and count the number of users who joined each year
number_users_joined_df = filtered_df.groupBy("join_year").agg(count("*").alias("number_users_joined"))



Find the median follower count of users have joined between 2015 and 2020.



In [0]:

# Join df_user_cleaned with df_pin based on the common column 'poster_name' to get the follower count information
joined_df = df_user.join(df_pin.select("ind", "follower_count"), on="ind", how="inner")

# Extract the year from the date_joined column
joined_df = joined_df.withColumn("join_year", year("date_joined"))

# Filter the DataFrame to select users who joined between 2015 and 2020
filtered_df = joined_df.filter((col("join_year") >= 2015) & (col("join_year") <= 2020))

# Calculate the median follower count for the selected users
median_follower_count_df = filtered_df.groupBy("join_year").agg(
    percentile_approx("follower_count", 0.5).alias("median_follower_count")
)



Find the median follower count of users that have joined between 2015 and 2020, based on  age group.



In [0]:
# Join df_user_cleaned with df_pin_cleaned based on the common column 'poster_name' to get the follower count information
joined_df = df_user_cleaned.join(df_pin_cleaned.select("ind", "follower_count"), on="ind", how="inner")

# Extract the year from the date_joined column
joined_df = joined_df.withColumn("join_year", year("date_joined"))

# Filter the DataFrame to select users who joined between 2015 and 2020
filtered_df = joined_df.filter((col("join_year") >= 2015) & (col("join_year") <= 2020))

# Categorize the age groups based on the original age column
filtered_df = filtered_df.withColumn("age_group",
    when((col("age") >= 18) & (col("age") <= 24), "18-24")
    .when((col("age") >= 25) & (col("age") <= 35), "25-35")
    .when((col("age") >= 36) & (col("age") <= 50), "36-50")
    .otherwise("+50")
)

# Calculate the median follower count for each age group
median_follower_count_df = filtered_df.groupBy("age_group", "join_year").agg(
    percentile_approx("follower_count", 0.5).alias("median_follower_count")
)

